In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import ipykernel
import pandas as pd

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)


In [2]:
import json
import random
import matplotlib.pyplot as plot
import numpy as np
import os
import jsonlines
import ast
import time, os, fnmatch, shutil
from collections import defaultdict
import matplotlib
import requests

import math
import random
import copy

from collections import namedtuple, deque

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

from games  import games
from games import games
from game   import Game
from player import Player_Template
from team   import Team
from userinput import userInput



HISTORY_FILE = None

summary_dict = {"losses":0,"wins":0,"draws":0}

cycle = 0

actions = [[], []]
rewards = [[], []]
history = {"cycle": cycle, "actions": actions, "rewards": rewards, "value": 0}

score_board = {}
#step_action_dict = defaultdict()
#step_action_dict['random_moves'] = defaultdict()
#step_action_dict['policy_moves'] = defaultdict()

numeric_names = {'w_R0': 1, 'w_K0': 2, 'w_B0': 3, 'w__K': 4, 'w__Q': 5, 'w_B1': 6, 'w_K1': 7, 'w_R1': 8, 'w_P0': 9, 'w_P1':
 10, 'w_P2': 11, 'w_P3': 12, 'w_P4': 13, 'w_P5': 14, 'w_P6': 15, 'w_P7': 16, 'b_P0': -16, 'b_P1': -15, 'b_P2': -14, 'b_P3':
 -13, 'b_P4': -12, 'b_P5': -11, 'b_P6': -10, 'b_P7': -9, 'b_R0': -8, 'b_K0': -7, 'b_B0': -6, 'b__K': -5, 'b__Q': -4, 'b_B1'
: -3, 'b_K1': -2, 'b_R1': -1}




In [3]:
%%writefile sparse_action_dict.py

sparse_action_dict = {'1,1,0':'010010011001','2,1,-2':'010011010111','2,1,2':'010011011011','2,2,-1':'010011100000','2,2,1':'010011100010','3,1,-1':'010100011000','3,1,1':'010100011010','4,1,-1':'010101011000','4,1,0':'010101011001','4,1,1':'010101011010','5,1,-1':'010110011000','5,1,0':'010110011001','5,1,1':'010110011010','6,1,-1':'010111011000','6,1,1':'010111011010','7,1,-2':'011000010111','7,1,2':'011000011011','7,2,-1':'011000100000','7,2,1':'011000100010','8,1,0':'011001011001','9,1,0':'011010011001','9,1,-1':'011010011000','9,1,1':'011010011010','10,1,0':'011011011001','10,1,-1':'011011011000','10,1,1':'011011011010','11,1,0':'011100011001','11,1,-1':'011100011000','11,1,1':'011100011010','12,1,0':'011101011001','12,1,-1':'011101011000','12,1,1':'011101011010','13,1,0':'011110011001','13,1,-1':'011110011000','13,1,1':'011110011010','14,1,0':'011111011001','14,1,-1':'011111011000','14,1,1':'011111011010','15,1,0':'100000011001','15,1,-1':'100000011000','15,1,1':'100000011010','16,1,0':'100001011001','16,1,-1':'100001011000','16,1,1':'100001011010','-16,-1,0':'000001001001','-16,-1,-1':'000001001000','-16,-1,1':'000001001010','-15,-1,0':'000010001001','-15,-1,-1':'000010001000','-15,-1,1':'000010001010','-14,-1,0':'000011001001','-14,-1,-1':'000011001000','-14,-1,1':'000011001010','-13,-1,0':'000100001001','-13,-1,-1':'000100001000','-13,-1,1':'000100001010','-12,-1,0':'000101001001','-12,-1,-1':'000101001000','-12,-1,1':'000101001010','-11,-1,0':'000110001001','-11,-1,-1':'000110001000','-11,-1,1':'000110001010','-10,-1,0':'000111001001','-10,-1,-1':'000111001000','-10,-1,1':'000111001010','-9,-1,0':'001000001001','-9,-1,-1':'001000001000','-9,-1,1':'001000001010','-8,-1,0':'001001001001','-7,-1,-2':'001010000111','-7,-1,2':'001010001011','-7,-2,-1':'001010000000','-7,-2,1':'001010000010','-6,-1,-1':'001011001000','-6,-1,1':'001011001010','-5,-1,-1':'001100001000','-5,-1,0':'001100001001','-5,-1,1':'001100001010','-4,-1,-1':'001101001000','-4,-1,0':'001101001001','-4,-1,1':'001101001010','-3,-1,-1':'001110001000','-3,-1,1':'001110001010','-2,-1,-2':'001111000111','-2,-1,2':'001111001011','-2,-2,-1':'001111000000','-2,-2,1':'001111000010','-1,-1,0':'010000001001'}

Overwriting sparse_action_dict.py


In [4]:
device  = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:0")
device

device(type='cuda', index=1)

In [5]:
Pieces = {
    'Pawn':("w_P","b_P"),
    'Knight':("w_K","b_K"),
    'King':("w__K","b__K"),
    'Queen':("w__Q","b__Q"),
    'Bishop':("w_B","b_B"),
    'Rook':("w_R","b_R"),
}
    

In [6]:
  def getBin(num):
        if int(num) != 0:
            return "{0:{fill}6b}".format(int(num)+17, fill='0')
        else:
            return "{0:{fill}6b}".format(0, fill='0')

In [7]:
Pieces_detail = defaultdict()
actions_array = []
sparse_action_dict = defaultdict()
actions = []

for piece in numeric_names.keys():
    action_sparse = []
    if piece[0] == 'w':
        idx = 0
    else:
        idx = 1
        
    if piece[1:3] == '_R':
        player_id = 'Rook'
    elif piece[1:3] == '_K':
        player_id = 'Knight'
    elif piece[1:3] == '_B':
        player_id = 'Bishop'
    elif piece[1:3] == '_P':
        player_id = 'Pawn'
    elif piece[1:4] == '__K':
        player_id = 'King'
    elif piece[:3] == '__Q':
        player_id = 'Queen'
        
    actions = games['chessy']['players'][player_id]['moves']
    
    actions = actions[idx]
    
    #action_diff = 4 - len(actions)
    
    #for i in range(action_diff):
    #    actions.append((0,0))
    
    piece_num_id = numeric_names[piece]
    
    piece_bin_id = '{0:06b}'.format(piece_num_id + 17)
    
    actions_ = []
    
    for action in actions:
        action_value_  = (8*action[0] + action[1] + 17)
        action_value   = '{0:06b}'.format(action_value_)
        actions_.append(action_value)
        
        piece_move_bin = (str(piece_bin_id)+str(action_value))
        
        actions_array.append(piece_move_bin)
        
        sparse_action  = str(piece_num_id)+","+str(action[0])+ "," +str(action[1])
        
        action_sparse.append(sparse_action)
        
        sparse_action_dict[sparse_action] = piece_move_bin
    
    print(piece_num_id, "\t",piece_bin_id, "\t", action_sparse,  "\t", actions, "\t", actions_)
    
    Pieces_detail[piece_num_id] = {
        'piece_num_id':piece_num_id,
        'piece_bin_id':piece_bin_id,
        'action_sparse':action_sparse,
        'actions_verbose': actions,
        'actions_bin': actions_
       }
    
action_ids = np.arange(0,len(sparse_action_dict.keys()),1)
action_ids

action_id_dict = {y:x for x,y in zip(action_ids,sparse_action_dict.keys())}
action_id_dict 

id_action_dict = {x:y for x,y in zip(action_ids,sparse_action_dict.keys())}
id_action_dict

1 	 010010 	 ['1,1,0'] 	 [(1, 0)] 	 ['011001']
2 	 010011 	 ['2,1,-2', '2,1,2', '2,2,-1', '2,2,1'] 	 [(1, -2), (1, 2), (2, -1), (2, 1)] 	 ['010111', '011011', '100000', '100010']
3 	 010100 	 ['3,1,-1', '3,1,1'] 	 [(1, -1), (1, 1)] 	 ['011000', '011010']
4 	 010101 	 ['4,1,-1', '4,1,0', '4,1,1'] 	 [(1, -1), (1, 0), (1, 1)] 	 ['011000', '011001', '011010']
5 	 010110 	 ['5,1,-1', '5,1,0', '5,1,1'] 	 [(1, -1), (1, 0), (1, 1)] 	 ['011000', '011001', '011010']
6 	 010111 	 ['6,1,-1', '6,1,1'] 	 [(1, -1), (1, 1)] 	 ['011000', '011010']
7 	 011000 	 ['7,1,-2', '7,1,2', '7,2,-1', '7,2,1'] 	 [(1, -2), (1, 2), (2, -1), (2, 1)] 	 ['010111', '011011', '100000', '100010']
8 	 011001 	 ['8,1,0'] 	 [(1, 0)] 	 ['011001']
9 	 011010 	 ['9,1,0', '9,1,-1', '9,1,1'] 	 [(1, 0), (1, -1), (1, 1)] 	 ['011001', '011000', '011010']
10 	 011011 	 ['10,1,0', '10,1,-1', '10,1,1'] 	 [(1, 0), (1, -1), (1, 1)] 	 ['011001', '011000', '011010']
11 	 011100 	 ['11,1,0', '11,1,-1', '11,1,1'] 	 [(1, 0), (1, -1), (1, 1)] 

{0: '1,1,0',
 1: '2,1,-2',
 2: '2,1,2',
 3: '2,2,-1',
 4: '2,2,1',
 5: '3,1,-1',
 6: '3,1,1',
 7: '4,1,-1',
 8: '4,1,0',
 9: '4,1,1',
 10: '5,1,-1',
 11: '5,1,0',
 12: '5,1,1',
 13: '6,1,-1',
 14: '6,1,1',
 15: '7,1,-2',
 16: '7,1,2',
 17: '7,2,-1',
 18: '7,2,1',
 19: '8,1,0',
 20: '9,1,0',
 21: '9,1,-1',
 22: '9,1,1',
 23: '10,1,0',
 24: '10,1,-1',
 25: '10,1,1',
 26: '11,1,0',
 27: '11,1,-1',
 28: '11,1,1',
 29: '12,1,0',
 30: '12,1,-1',
 31: '12,1,1',
 32: '13,1,0',
 33: '13,1,-1',
 34: '13,1,1',
 35: '14,1,0',
 36: '14,1,-1',
 37: '14,1,1',
 38: '15,1,0',
 39: '15,1,-1',
 40: '15,1,1',
 41: '16,1,0',
 42: '16,1,-1',
 43: '16,1,1',
 44: '-16,-1,0',
 45: '-16,-1,-1',
 46: '-16,-1,1',
 47: '-15,-1,0',
 48: '-15,-1,-1',
 49: '-15,-1,1',
 50: '-14,-1,0',
 51: '-14,-1,-1',
 52: '-14,-1,1',
 53: '-13,-1,0',
 54: '-13,-1,-1',
 55: '-13,-1,1',
 56: '-12,-1,0',
 57: '-12,-1,-1',
 58: '-12,-1,1',
 59: '-11,-1,0',
 60: '-11,-1,-1',
 61: '-11,-1,1',
 62: '-10,-1,0',
 63: '-10,-1,-1',
 64: '-10,

In [8]:
args = { 
    "BUFFER_SIZE":int(1e6),
    "BATCH_SIZE":1024,  
    "GAMMA":0.99,
    "TAU":2e-3,
    "LR_ACTOR":1e-3,
    "LR_CRITIC":1.1e-3,
    "WEIGHT_DECAY":0.0001,
    "UPDATE_EVERY":5,
    "EXPLORE_NOISE":0.05,
    "FC1_UNITS":4096,
    "FC2_UNITS":10240,
    "FC3_UNITS":1024,
    "seed":0,
    "state_size":384,
    "action_size":87,
    "num_agents":2,
    "device":device,
    'mcritic_path':'/home/ubuntu/chessy/checkpoint_mCritic.pth',
    'agent_p0_path':'/home/ubuntu/chessy/checkpoint_p0.pth',
    'agent_p1_path':'/home/ubuntu/chessy/checkpoint_p1.pth',
    'action_ids': action_ids,
    'action_id_dict': action_id_dict,
    'id_action_dict': id_action_dict,
}

In [9]:
#sparse_action_dict

In [10]:
initial_state =  "010010010011010100010101010110010111011000011001011010011011011100011101011110011111100000100001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000010000011000100000101000110000111001000001001001010001011001100001101001110001111010000"

In [11]:
best_moves   = 0
policy_moves = 0
random_moves = 0

In [12]:
def getFunc(piece):
    def piece_func():
        return piece
    return piece_func()

def Bishop():
    return getFunc("Bishop")

def Pawn():
    return getFunc("Pawn")

def King():
    return getFunc("King")

def Queen():
    return getFunc("Queen")

def Knight():
    return getFunc("Knight")

def Rook():
    return getFunc("Rook")


In [13]:

class Incorrect_Input_error(Exception):
  """Generic input error handler: raised in the case that any of the user inputed data is incorrect"""
  pass



In [14]:
def run_trial(user_input, env):
    global cycle
    for j in range(user_input.num_sides):
        team_name = "team_" + str(j)
        team_name = Team(user_input.game, user_input.teams[j]["team_name"], j,user_input.teams[j]["skill"] / 10, user_input.teams[j]["strategy"])

        env.insert_team(team_name)
            
    env.not_deadlocked = True
    env.states.append(initial_state)
    time_step = 0
        
    while env.not_deadlocked:
        #print("ENV.STATES[-1]:\n{}\n".format(env.states[-1]))
        state_prior = env.states[-1] 
        state_prior_bin   = list(map(int, state_prior))
        #print("STATE_PRIOR_BIN:\n{}\n".format(state_prior_bin))
        state_prior_tensor = torch.IntTensor(state_prior_bin).to(device)
        actions_from_model = agent_p0.actor_network(state_prior_tensor.float())
        action_from_model  = np.argmax(actions_from_model.detach().cpu())
        #print("ACTION_FROM_MODEL:\n{}\n".format(action_from_model))
        #print("\n=====\n"*8)
        env.step(cycle, user_input)
        next_state = ""
        state_prime = ""
        if time_step % 2 == 0:
            state_prime = env.states[-1]
        else:
            state_prime = env.states[-1]
            
        
        #print('FEASIBLE_MOVES:\n{}\n'.format(env.feasible_moves_))
        print("CYCLE:\t{}\nSTATE_PRIOR:\t{}\nACTION:\t{}\nNEXT_STATE:\t{}\nREWARD:\t{}\n".format(cycle,state_prior,env.last_action_sparse, next_state, env.last_reward))
            
        time_step += 1

In [15]:
env = None

In [16]:
def run_trials(user_input):
    """ Runs the num_trials """
    global cycle
    global HISTORY_FILE
    global SUMMARY_FILE
    global env
    
    print("==============================RUN_TRIALS==================================")
    
    t = time.localtime()
    timestamp = time.strftime('%b_%d_%Y_%H%M', t)
    num_trials = user_input.num_trials
    num_sides = user_input.num_sides
    
    SUMMARY_FILE = ("/data_data/reinforcement_learning/results/summary_file.tsv")

    HISTORY_FILE = ("/data_data/reinforcement_learning/results/history_file_" + str(num_trials) + "_trials_" + str(num_sides) + "_sides_" + str(timestamp))
    
    """ Clear output file """
    
    with open(HISTORY_FILE, "w") as history_file:
        history_file.write("\n")
        
        
    for i in range(num_trials):
        
        sides = [x for x in range(num_sides)]
        
        env = Game(user_input.game, 8, sides, user_input.display_board_positions)

        run_trial(user_input,env)

        cycle += 1
    
   


In [17]:
args['action_size']

87

In [18]:
from agent import Agent
from model import ActorNetwork, CriticNetwork, MCritic
from replaybuffer import ReplayBuffer

mCritic       =  MCritic(args['state_size'],args['action_size'], args)
agent_p0      =  Agent(args['state_size'],args['action_size'], 0,args)
agent_p1      =  Agent(args['state_size'],args['action_size'], 1,args)

In [19]:
print(mCritic.network)
print(agent_p0.actor_network)

CriticNetwork(
  (fc1): Linear(in_features=471, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=10240, bias=True)
  (fc3): Linear(in_features=10240, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1, bias=True)
)
ActorNetwork(
  (fc1): Linear(in_features=384, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=10240, bias=True)
  (fc3): Linear(in_features=10240, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=87, bias=True)
)


In [20]:

from agent import Agent
from model import ActorNetwork, CriticNetwork, MCritic
from replaybuffer import ReplayBuffer

#def main():
if __name__ == "__main__":
    global cycle
    global HISTORY_FILE
    global score_board
    score_board[0] = []
    score_board[1] = []
    user_input = userInput()
    run_trials(user_input)


#main()


select a game: ['chessy' or 'checkers']: chessy
select number of teams:  [0, 1 or 2] 2
Choose a name for this team [e.g. 'blue_team' or 'green_team']: Agent
Choose a color for team_2 [e.g. 'blue': green
Please enter team's skill_level [1 = Novice, 10 = expert]: 10
Please enter team's strategy [0 = 'cooperative', 1 = 'competitive']: 1
Choose a name for this team [e.g. 'blue_team' or 'green_team']: Environment
Choose a color for team_2 [e.g. 'blue': red
Please enter team's skill_level [1 = Novice, 10 = expert]: 1
Please enter team's strategy [0 = 'cooperative', 1 = 'competitive']: 1
How many trials would you like to run? [1 - 1,000,000] 1
Do you want to see the board positions in realtime? [ 'Yes' or 'No' ]no
==============================RUN_TRIALS==================================


/home/ubuntu/chessy/model.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.fc4(x))


CYCLE:	0
STATE_PRIOR:	010010010011010100010101010110010111011000011001011010011011011100011101011110011111100000100001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000010000011000100000101000110000111001000001001001010001011001100001101001110001111010000
ACTION:	12,1,-1
NEXT_STATE:	
REWARD:	-1

CYCLE:	0
STATE_PRIOR:	000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ACTION:	-2,-2,-1
NEXT_STATE:	
REWARD:	-1

CYCLE:	0
STATE_PRIOR:	000000000000000000000000000000000000000000000000000000000000000000000000000000000

CYCLE:	0
STATE_PRIOR:	000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ACTION:	-12,-1,1
NEXT_STATE:	
REWARD:	-1

CYCLE:	0
STATE_PRIOR:	000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ACTION:	8,1,0
NEXT_STATE:	
REWARD:	-1

CYCLE:	0
STATE_PRIOR:	00000000000000000000000000000000000000000000000000000000000000000000000000000000000

CYCLE:	0
STATE_PRIOR:	000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ACTION:	-13,-1,1
NEXT_STATE:	
REWARD:	-1

CYCLE:	0
STATE_PRIOR:	000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ACTION:	1,1,0
NEXT_STATE:	
REWARD:	-1

CYCLE:	0
STATE_PRIOR:	00000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [21]:
env.states

['...']

In [22]:
args['action_size']

87

In [23]:
test_state = torch.IntTensor(np.random.choice((0,1),args['state_size'])).to(device)
test_state

tensor([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,

In [24]:
test_action = torch.IntTensor([34]).to(device)
test_action[:25]

tensor([34], device='cuda:1', dtype=torch.int32)

In [25]:
test_action = torch.IntTensor(np.array([34]).reshape((1,))).to(device)
test_action[:25]

tensor([34], device='cuda:1', dtype=torch.int32)

In [26]:
x = torch.cat((test_state, test_action))
print(x[:25])

tensor([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0], device='cuda:1', dtype=torch.int32)


In [27]:
test_state.shape

torch.Size([384])

In [28]:
test_action = agent_p0.actor_network(test_state.float())
print(test_action)

tensor([0.0116, 0.0113, 0.0118, 0.0116, 0.0119, 0.0114, 0.0115, 0.0113, 0.0116,
        0.0117, 0.0112, 0.0114, 0.0112, 0.0118, 0.0117, 0.0117, 0.0116, 0.0113,
        0.0117, 0.0114, 0.0114, 0.0117, 0.0112, 0.0118, 0.0112, 0.0112, 0.0113,
        0.0112, 0.0118, 0.0112, 0.0117, 0.0114, 0.0115, 0.0113, 0.0118, 0.0111,
        0.0116, 0.0115, 0.0116, 0.0116, 0.0112, 0.0115, 0.0114, 0.0115, 0.0117,
        0.0118, 0.0118, 0.0114, 0.0113, 0.0116, 0.0114, 0.0118, 0.0112, 0.0118,
        0.0112, 0.0118, 0.0112, 0.0111, 0.0115, 0.0117, 0.0117, 0.0118, 0.0113,
        0.0114, 0.0114, 0.0113, 0.0116, 0.0114, 0.0113, 0.0113, 0.0116, 0.0113,
        0.0118, 0.0116, 0.0118, 0.0116, 0.0112, 0.0118, 0.0113, 0.0113, 0.0114,
        0.0116, 0.0115, 0.0117, 0.0112, 0.0117, 0.0113], device='cuda:1',
       grad_fn=<SoftmaxBackward>)


In [29]:
np.argmax(test_action.detach().cpu())

tensor(4)

In [30]:
test_action

tensor([0.0116, 0.0113, 0.0118, 0.0116, 0.0119, 0.0114, 0.0115, 0.0113, 0.0116,
        0.0117, 0.0112, 0.0114, 0.0112, 0.0118, 0.0117, 0.0117, 0.0116, 0.0113,
        0.0117, 0.0114, 0.0114, 0.0117, 0.0112, 0.0118, 0.0112, 0.0112, 0.0113,
        0.0112, 0.0118, 0.0112, 0.0117, 0.0114, 0.0115, 0.0113, 0.0118, 0.0111,
        0.0116, 0.0115, 0.0116, 0.0116, 0.0112, 0.0115, 0.0114, 0.0115, 0.0117,
        0.0118, 0.0118, 0.0114, 0.0113, 0.0116, 0.0114, 0.0118, 0.0112, 0.0118,
        0.0112, 0.0118, 0.0112, 0.0111, 0.0115, 0.0117, 0.0117, 0.0118, 0.0113,
        0.0114, 0.0114, 0.0113, 0.0116, 0.0114, 0.0113, 0.0113, 0.0116, 0.0113,
        0.0118, 0.0116, 0.0118, 0.0116, 0.0112, 0.0118, 0.0113, 0.0113, 0.0114,
        0.0116, 0.0115, 0.0117, 0.0112, 0.0117, 0.0113], device='cuda:1',
       grad_fn=<SoftmaxBackward>)

In [31]:
mCritic.network(test_state.float(), test_action.float())

tensor([-0.0024], device='cuda:1', grad_fn=<AddBackward0>)

In [32]:
torch.cat((test_state.float(), test_action.float()))

tensor([0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000,
        0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [33]:
!ls 

 1
 agent.py
 checkpoint_mCritic.pth
 checkpoint_p0.pth
 checkpoint_p1.pth
 Chessy.ipynb
 chessy.json
 chessy_v2.05.py
 chessy_v2.1.py
'DRL with Spark and PyTorch.ipynb'
 error_handling.py
 game.py
 games.py
 main.py
 model.py
 noise.py
 player.py
 plotutils.py
 __pycache__
 README.md
'Reinforcement Learning - Chessy Using Hadoop MapReduce.ipynb'
 replaybuffer.py
 simple.json
 sparse_action_dict.py
 team.py
 userinput.py
 utils.py


In [34]:
print(numeric_names)

{'w_R0': 1, 'w_K0': 2, 'w_B0': 3, 'w__K': 4, 'w__Q': 5, 'w_B1': 6, 'w_K1': 7, 'w_R1': 8, 'w_P0': 9, 'w_P1': 10, 'w_P2': 11, 'w_P3': 12, 'w_P4': 13, 'w_P5': 14, 'w_P6': 15, 'w_P7': 16, 'b_P0': -16, 'b_P1': -15, 'b_P2': -14, 'b_P3': -13, 'b_P4': -12, 'b_P5': -11, 'b_P6': -10, 'b_P7': -9, 'b_R0': -8, 'b_K0': -7, 'b_B0': -6, 'b__K': -5, 'b__Q': -4, 'b_B1': -3, 'b_K1': -2, 'b_R1': -1}


In [35]:
actions_array

['010010011001',
 '010011010111',
 '010011011011',
 '010011100000',
 '010011100010',
 '010100011000',
 '010100011010',
 '010101011000',
 '010101011001',
 '010101011010',
 '010110011000',
 '010110011001',
 '010110011010',
 '010111011000',
 '010111011010',
 '011000010111',
 '011000011011',
 '011000100000',
 '011000100010',
 '011001011001',
 '011010011001',
 '011010011000',
 '011010011010',
 '011011011001',
 '011011011000',
 '011011011010',
 '011100011001',
 '011100011000',
 '011100011010',
 '011101011001',
 '011101011000',
 '011101011010',
 '011110011001',
 '011110011000',
 '011110011010',
 '011111011001',
 '011111011000',
 '011111011010',
 '100000011001',
 '100000011000',
 '100000011010',
 '100001011001',
 '100001011000',
 '100001011010',
 '000001001001',
 '000001001000',
 '000001001010',
 '000010001001',
 '000010001000',
 '000010001010',
 '000011001001',
 '000011001000',
 '000011001010',
 '000100001001',
 '000100001000',
 '000100001010',
 '000101001001',
 '000101001000',
 '000101001010

In [36]:
Pieces_detail

defaultdict(None,
            {1: {'piece_num_id': 1,
              'piece_bin_id': '010010',
              'action_sparse': ['1,1,0'],
              'actions_verbose': [(1, 0)],
              'actions_bin': ['011001']},
             2: {'piece_num_id': 2,
              'piece_bin_id': '010011',
              'action_sparse': ['2,1,-2', '2,1,2', '2,2,-1', '2,2,1'],
              'actions_verbose': [(1, -2), (1, 2), (2, -1), (2, 1)],
              'actions_bin': ['010111', '011011', '100000', '100010']},
             3: {'piece_num_id': 3,
              'piece_bin_id': '010100',
              'action_sparse': ['3,1,-1', '3,1,1'],
              'actions_verbose': [(1, -1), (1, 1)],
              'actions_bin': ['011000', '011010']},
             4: {'piece_num_id': 4,
              'piece_bin_id': '010101',
              'action_sparse': ['4,1,-1', '4,1,0', '4,1,1'],
              'actions_verbose': [(1, -1), (1, 0), (1, 1)],
              'actions_bin': ['011000', '011001', '011010']},


In [37]:
print(actions)

[(-1, 0)]


In [38]:
'{0:06b}'.format(32)

'100000'

In [39]:
getBin(1)

'010010'

In [40]:
!cat games.py


games = {
  "chessy": {
    "players": {
      "Pawn": {
        "name": "p",
        "start_pos": [(1, 0), (6, 0)],
        "moves": [[(1, 0), (1, -1), (1, 1)], [(-1, 0), (-1, -1), (-1, 1)]],
        "number": 8,
        "value": 1,
        "image": [u"\u265F", u"\u2659"]
      },
      "Knight": {
        "name":
        "K",
        "start_pos": [[(0, 1), (7, 1)], [(0, 6), (7, 6)]],
        "moves": [[(1, -2), (1, 2), (2, -1), (2, 1)], [(-1, -2), (-1, 2),
                                                       (-2, -1), (-2, 1)]],
        "number":
        2,
        "value":
        3,
        "image": [u"\u265E", u"\u2658"]
      },
      "King": {
        "name": "_K",
        "start_pos": [(0, 3), (7, 3)],
        "moves": [[(1, -1), (1, 0), (1, 1)], [(-1, -1), (-1, 0), (-1, 1)]],
        "number": 1,
        "value": 20,
        "image": [u"\u265A", u"\u2654"]
      },
      "Queen": {
        "name": "_Q",
        "start_pos": [(0, 4), (7, 4)],
        "moves": [[(1, -1), (1, 